In [1]:
import requests
import json
import time
import numpy as np
import pandas as pd
import os
import re

In [2]:
# Папка где исксть таблицы с комментами
storage_dir_name = "RawComments"

# Файл откуда подгружать список видео
file_to_analyze_name = "videos_to_analyze.csv"

# Файл куда сохранять отфильтрованное
file_to_analyze_filtered_name = "videos_to_analyze_filtered.csv"

# Столбцы таблиц с видео
column_names_video = ["id", 
                      "title",  
                      "publishedAt", 
                      "channelTitle", 
                      "viewCount", 
                      "likeCount", 
                      "dislikeCount", 
                      "commentCount"]
# Столбцы таблиц с комментами
column_names_comment = ["id",
                        "videoId",
                        "authorVideoName",
                        "totalReplyCount",
                        "likeCount",
                        "authorDisplayName",
                        "textDisplay",
                        "publishedAt", "isReply"]

In [3]:
# Получаем списки скачанных видео (регуляркой по папке) и списки видео для анализа (из таблицы)
df_fvideos = pd.read_csv(file_to_analyze_name)[column_names_video]

comment_files = [storage_dir_name + """/""" + file_name for file_name in os.listdir(storage_dir_name) 
                 if file_name.find("_video_comments.csv") >= 0]

existed_v_ids = [re.split(storage_dir_name + r'\/(.*)_video_comments.csv', comment_file)[1] 
                         for comment_file in comment_files]

In [4]:
# Сравним количество скачанных комментариев с цифрой из таблицы общих видеозаписей
loaded_comments = [[re.split(storage_dir_name + r'\/(.*)_video_comments.csv', comment_file)[1], 
                    pd.read_csv(comment_file)[column_names_comment].shape[0]] for comment_file in comment_files]

comments_comparisons = []
for loaded_comment in loaded_comments:
    if not df_fvideos[df_fvideos["id"] == loaded_comment[0]]["commentCount"].empty:
        comments_comparisons.append(loaded_comment + [df_fvideos[df_fvideos["id"] == loaded_comment[0]]["commentCount"].array[0]])
    

unconsistent_filter = np.array([min(comments_comparison[1], comments_comparison[2]) / max(comments_comparison[1], comments_comparison[2]) 
                                for comments_comparison in comments_comparisons]) < 0.9

unconsistent_videos = np.array(comments_comparisons)[unconsistent_filter]

unconsistent_videos

array([], shape=(0, 3), dtype='<U11')

In [5]:
# Удалим из датафрейма то, что лишнее
for unconsistent_video in unconsistent_videos:
    df_fvideos = df_fvideos[df_fvideos["id"] != unconsistent_video[0]]

In [6]:
df_fvideos.shape

(205, 8)

In [7]:
df_fvideos.to_csv(file_to_analyze_filtered_name)